<a href="https://colab.research.google.com/github/nishinishic/sca-1-cp/blob/main/notebooks/pipeline_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>